In [1]:
import numpy as np
import pandas as pd

In [3]:
d = {'Clubes': [], 'PG': [], 'j': [], 'V': [], 'E': [], 'D': [], 'GP': [], 'GC': [], 'SG': []}
dfClassChampionship = pd.DataFrame(data=d);
dfDataChampionship = pd.read_excel("b2016.xlsx", encoding='utf8')

In [4]:
dfDataChampionship.head()

,Horario,Dia,Data,Clube 1,Clube 2,Placar,Vencedor,Rodada,Arena
0,16:00,Sábado,14/05/2016,Flamengo,Sport,1 x 0,Flamengo,1ª Rodada,Raulino de Oliveira
1,16:00,Sábado,14/05/2016,Palmeiras,Atlético-PR,4 x 0,Palmeiras,1ª Rodada,Palestra Ítalia
2,18:30,Sábado,14/05/2016,Atlético-MG,Santos,1 x 0,Atlético-MG,1ª Rodada,Independência
3,21:00,Sábado,14/05/2016,Coritiba,Cruzeiro,1 x 0,Coritiba,1ª Rodada,Couto Pereira
4,11:00,Domingo,15/05/2016,Santa Cruz,Vitória,4 x 1,Santa Cruz,1ª Rodada,Arruda


** Get all clubes for championship **



In [5]:
#Cancatenando as colunas Clube 1 e Clube 2
clubs  =  pd.concat([dfDataChampionship['Clube 1'].str.lower(), dfDataChampionship['Clube 2'].str.lower()], axis=1, keys=['Clubes'])

In [6]:
#Obtem todos os clubes 
cb = pd.DataFrame({'Clubes': clubs['Clubes'].unique()})

In [7]:
#Todos os clubes que participaram dp campeonato
cb

,Clubes
0,flamengo
1,palmeiras
2,atlético-mg
3,coritiba
4,santa cruz
5,botafogo
6,corinthians
7,américa-mg
8,figueirense
9,internacional


In [8]:
def getNumberWinner(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter = df["Vencedor"].str.lower() == clube
    return (data[filter]['Vencedor'].count()).astype(np.int64)

def getNumberDepartures(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    return (data[filter1]['Clube 1'].count() + data[filter2]['Clube 2'].count()).astype(np.int64)

def getPoints(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    filter3 = df["Vencedor"].str.lower() == clube
    filter4 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter5 = df["Vencedor"].str.lower() == '-'
    
    v1 = data[(filter1) & (filter3)]
    v1 = v1['Vencedor'].count()
    v2 = data[(filter2) & (filter3)]
    v2 = v2['Vencedor'].count() 
    v3 = data[(filter4) & (filter5)]
    v3 = v3['Vencedor'].count()
    return ((v2*3)+(v1*3)+v3).astype(np.int64)

def getDraw(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = df["Vencedor"].str.lower() == '-'
    df = data[(filter1) & (filter2)]
    empates = df['Vencedor'].count()
    return empates.astype(np.int64)

def getDefeats(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = (df["Vencedor"].str.lower() != clube) & (df["Vencedor"].str.lower() != '-')
    df = data[(filter1) & (filter2)]
    derrotas = df['Vencedor'].count()
    return derrotas.astype(np.int64)

def getGP(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1  =  df["Clube 1"].str.lower() == clube
    filter2  =  df["Clube 2"].str.lower() == clube
    df1      =  data[(filter1)]
    df2      =  data[(filter2)]
    placar1  =  df1['Placar'].str.split('x')
    placar2  =  df2['Placar'].str.split('x')
    gp       =  0
    gc       =  0
    for g1, g2 in placar1:
        gp = (gp + pd.to_numeric( g1 ))
        gc = (gc + pd.to_numeric( g2 ))
            
    for g1, g2 in placar2:
        gp = (gp + pd.to_numeric( g2 )) 
        gc = (gc + pd.to_numeric( g1 ))
    return gp, gc

In [9]:
dfTable = cb[["Clubes"]].copy()
for column in ["PG", "J", "V", "E", "D", "GP", "GC", "SG"]:
    dfTable[column] = pd.np.nan
    
    
for index, row in dfTable.iterrows():
    c1  =  row['Clubes']
    if type(c1) != unicode:
        c1 =  c1.decode('utf-8')
    c1 =  c1.strip()
    
    pg       =  getPoints(dfDataChampionship, c1 )
    j        =  getNumberDepartures(dfDataChampionship, c1 )
    v        =  getNumberWinner(dfDataChampionship, c1 )
    e        =  getDraw(dfDataChampionship, c1 )
    d        =  getDefeats(dfDataChampionship, c1 )
    gp, gc   =  getGP(dfDataChampionship, c1 )
    
    
    dfTable.at[index, 'PG'] = pg
    dfTable.at[index, 'J']  = j
    dfTable.at[index, 'V']  = v
    dfTable.at[index, 'E']  = e
    dfTable.at[index, 'D']  = d
    dfTable.at[index, 'GP'] = gp
    dfTable.at[index, 'GC'] = gc
    dfTable.at[index, 'SG'] = gp - gc
    
    #print c1

dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
0,flamengo,71.0,38.0,20.0,11.0,7.0,52.0,35.0,17.0
1,palmeiras,80.0,38.0,24.0,8.0,6.0,62.0,32.0,30.0
2,atlético-mg,63.0,38.0,17.0,12.0,9.0,61.0,50.0,11.0
3,coritiba,46.0,38.0,11.0,13.0,14.0,41.0,42.0,-1.0
4,santa cruz,31.0,38.0,8.0,7.0,23.0,45.0,69.0,-24.0
5,botafogo,59.0,38.0,17.0,8.0,13.0,43.0,39.0,4.0
6,corinthians,55.0,38.0,15.0,10.0,13.0,48.0,42.0,6.0
7,américa-mg,28.0,38.0,7.0,7.0,24.0,23.0,58.0,-35.0
8,figueirense,37.0,38.0,8.0,13.0,17.0,30.0,50.0,-20.0
9,internacional,43.0,38.0,11.0,10.0,17.0,35.0,41.0,-6.0


In [10]:
reorderlist = [   "Palmeiras", "Santos", "Flamengo", "Atlético-MG", "Botafogo",
                  "Atlético-PR", "Corinthians", "Ponte Preta", "Grêmio", "São Paulo",
                  "Chapecoense", "Cruzeiro", "Fluminense", "Sport", "Coritiba", "Vitória",
                  "Internacional", "Figueirense", "Santa Cruz", "América-MG"
              ]
listIndex   = []

#Processa a lista para gerar a lista de ordenação
for clube in reorderlist:
    for index, row in dfTable.iterrows():
        #Verifica se o valor de clube é um unicode, se não, convete isso.
        if type(clube) != unicode:
            c =  clube.decode('utf-8')
        else:
            c =  clube
        
        c1  =  row['Clubes']
        #print c1
        if type(c1) != unicode:
            c1 =  c1.decode('utf-8')
        
        c1 =  c1.strip()
        c  =  c.lower()
        
        #Verifica se o clube iterado é igual ao clube da lista, se sim
        #retorna seu indice
        if c1 == c:
            listIndex.append( index )
            
#Reordena a lista
dfTable  =  dfTable.reindex(index=listIndex, columns=["Clubes", "PG", "J", "V", "E", "D", "GP", "GC", "SG"])
#Com a lista ordenada, corrige os indices.
dfTable.index =  np.arange(1, len(dfTable)+1)
dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
1,palmeiras,80.0,38.0,24.0,8.0,6.0,62.0,32.0,30.0
2,santos,71.0,38.0,22.0,5.0,11.0,59.0,35.0,24.0
3,flamengo,71.0,38.0,20.0,11.0,7.0,52.0,35.0,17.0
4,atlético-mg,63.0,38.0,17.0,12.0,9.0,61.0,50.0,11.0
5,botafogo,59.0,38.0,17.0,8.0,13.0,43.0,39.0,4.0
6,atlético-pr,57.0,38.0,17.0,6.0,15.0,38.0,32.0,6.0
7,corinthians,55.0,38.0,15.0,10.0,13.0,48.0,42.0,6.0
8,ponte preta,53.0,38.0,15.0,8.0,15.0,48.0,52.0,-4.0
9,grêmio,53.0,38.0,14.0,11.0,13.0,41.0,44.0,-3.0
10,são paulo,52.0,38.0,14.0,10.0,14.0,44.0,36.0,8.0


** Por algum motivo que desconheço, na fonte dos dados, os clubes Atlético-PR possui 57 pontos, Chapecoense 52 pontos, mas na soma dos jogos apresentam números diferentes **